In [ ]:
import json 
  
  
 #the file to be converted to json format 
filename = 'data.txt'
  
# dictionary where the lines from 
# text will be stored 
dict1 = {} 
  
# creating dictionary 
with open(filename) as fh: 
  
    for line in fh: 
  
        # reads each line and trims of extra the spaces  
        # and gives only the valid words 
        command, description = line.strip().split(None, 1) 
  
        dict1[command] = description.strip() 
  #creating json file 
# the JSON file is named as test1 
out_file = open("test.json", "w") 
json.dump(dict1, out_file, indent = 4, sort_keys = False) 
out_file.close() 

In [ ]:
import spacy
import random


TRAIN_DATA = [("This product includes code licensed under the GNU General Public License, the GNU Lesser General Public\r",{"entities":[[22,26,"TECH"],
[46,49,"TECH"]]}),
 ("\fSummary of the Server Configuration Page\r",{"entities":[[16,22,"TECH"]]}),
("Generating an Example of Import File XML Format\r",{"entities":[[37,40,"TECH"]]}),
("Configuring ClearPass as an RFC 3576 (CoA) Server\r",{"entities":[[9,32,"TECH"],[38,41,"TECH"]]}),
("Configuring the ClearPass Server as a CoA Server\r",{"entities":[[38,48,"TECH"]]}),
("Using the CLI\r",{"entities":[[10,13,"TECH"]]}),
("Adding an SSID to the Mobility Controller for 802.1X Authentication\r",{"entities":[[10,14,"TECH"]]})]
def train_spacy(data,iterations):
    TRAIN_DATA = data
    nlp = spacy.blank('en')  # create blank Language class
    # create the built-in pipeline components and add them to the pipeline
    # nlp.create_pipe works for built-ins that are registered with spaCy
    if 'ner' not in nlp.pipe_names:
        ner = nlp.create_pipe('ner')
        nlp.add_pipe(ner, last=True)
       

    # add labels
    for _, annotations in TRAIN_DATA:
         for ent in annotations.get('entities'):
            ner.add_label(ent[2])

    # get names of other pipes to disable them during training
    other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
    with nlp.disable_pipes(*other_pipes):  # only train NER
        optimizer = nlp.begin_training()
        for itn in range(iterations):
            print("Statring iteration " + str(itn))
            random.shuffle(TRAIN_DATA)
            losses = {}
            for text, annotations in TRAIN_DATA:
                nlp.update(
                    [text],  # batch of texts
                    [annotations],  # batch of annotations
                    drop=0.2,  # dropout - make it harder to memorise data
                    sgd=optimizer,  # callable to update weights
                    losses=losses)
            print(losses)
    return nlp

In [ ]:
prdnlp = train_spacy(TRAIN_DATA, 20)

# Save our trained Model
modelfile = input("Enter your Model Name: ")
prdnlp.to_disk(modelfile)

In [1]:
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
import io

In [5]:
def pdfparser(data):

    fp = open(data, 'rb')
    rsrcmgr = PDFResourceManager() #to store shared resources such as fonts or images
    retstr = io.StringIO() # Cast to StringIO object
    codec = 'utf-8'
    # Set parameters for analysis.
    laparams = LAParams()
    # Create a PDF device object
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    # Create a PDF interpreter object.
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    # Process each page contained in the document.
    with open('extracted.txt','w') as f:
        for page in PDFPage.get_pages(fp):
            interpreter.process_page(page)
            data =  retstr.getvalue()
        f.write(data)
    print("done")


In [ ]:
pdffile = input("Enter the .pdf file name")
pdfparser(pdffile)

In [1]:
import spacy
prdnlp = spacy.load("myspacy")

In [ ]:
with open('extracted.txt','r') as f:
    content1=f.read()
    print(content1)
    content=content1.split(' ')
    print(content)
doc = prdnlp(content1)
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

In [ ]:
from spacy import displacy
displacy.render(doc, jupyter=True, style='ent')

In [22]:
items = [x.text for x in doc.ents]

In [4]:
import csv

words=[]
tech_words=[]
versions=[]
file='kb - Sheet1.csv'.encode("utf-8")

with open(file, 'r') as csvfile:
    reader = csv.reader((line.replace('\0','') for line in csvfile), delimiter=",",skipinitialspace=True)
    for row in reader:
        words.extend(row)

In [23]:
#print(type(content))
for i in range(len(content)):
    #print(content[i])
    if content[i] in words:
        tech_words.append(content[i])
        #print(tech_words)
        for n in content[i+1].split('.'):
            if n in ['0','1','2','3','4','5','6','7','8','9']:
                versions.append(content[i+1])
                break
            elif content[i+1]=='ver' or content[i+1]=='version':
                versions.append(content[i+2])
                break
            else:
                versions.append('NA')
                break

In [18]:
def ans_query(q):
    if q.lower()=="are there any technologies used?":
        if tech_words:
            print('Yes')
        else:
            print('No technology is used')
    elif q.lower()=="what are the technologies used?":
        print(tech_words)
    else:
        for i in range(len(versions)):
            print(tech_words[i]," - ", versions[i])

In [24]:
choice=1
while choice:
    query=input("Ask me anything: ")
    ans_query(query)
    choice=int(input("have more questions? [yes=1|no=0]"))

Ask me anything: are there any technologies used?
Yes
have more questions? [yes=1|no=0]1
Ask me anything: what are the technologies used?
['AI', 'ML', 'python', 'Server', 'CLI', 'XML', 'Server', 'Server', 'Server', 'CoA', 'Server', 'CoA', 'SSID', 'SSID', 'CLI', 'Server', 'Server', 'Server', 'Server', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'VLAN', 'RADIUS', 'Server', 'RADIUS', 'Aruba', 'Aruba', 'Aruba', 'Aruba', 'DHCP', 'Server', 'Server', 'Aruba', 'RADIUS', 'CLI', 'Aruba', 'Server', 'CLI', 'Server', 'Server', 'Server', 'Server', 'Server', 'Server', 'Server', 'Server', 'CLI', 'CLI', 'CLI', 'Aruba', 'Server', 'Server', 'Server', 'Server', 'SSID', 'XML', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'CoA', 'CoA', 'XML', 'XML', 'XML', 'XML', 'XML', 'Server', 'RADIUS', 'LAN', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'Server', 'RADIUS', 'Server', 'RADIUS', 'RADIUS', 'Server', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'RADIUS', 'Server', 'RA

RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
RADIUS  -  NA
Aruba  -  NA
RADIUS  -  NA
Server  -  NA
Server  -  NA
Server  -  NA
Server  -  NA
Server  -  NA
Server  -  NA
Aruba  -  NA
Aruba  -  NA
RADIUS  -  NA
NAS  -  NA
NAS  -  NA
RADIUS  -  NA
RADIUS  -  NA
LAN  -  NA
DHCP  -  NA
VLAN  -  NA
VLAN  -  NA
RADIUS  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
RADIUS  -  NA
RADIUS  -  NA
VLAN  -  NA
RADIUS  -  NA
DHCP  -  NA
DHCP  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  -  NA
VLAN  - 